In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the GROQ_API_KEY from environment variables
groq_api_key = os.getenv("GROQ_API_KEY_1")

In [ ]:
loader = TextLoader("health.txt")
documents = loader.load()
documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

In [ ]:
docs = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in docs]

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

db = FAISS.from_documents(
    embedding=embeddings,
    documents=docs
    )

In [ ]:
query = "I have experiencing  with sleep disorders"
test_search = db.similarity_search(query)

In [ ]:
search_results_content = " ".join([doc.page_content for doc in test_search])
search_results_content

In [ ]:
# Define the system and human messages
system = "You are a Fitness Coach and you will answer based on hte information i give you."
human = "{text}"

# Create the chat prompt template
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

# Initialize the chat model with specific parameters
chat = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

# Combine the prompt and chat into a chain
chain = prompt | chat

# Define the query for similarity search
query = "I have experiencing  with sleep disorders"

# Perform the similarity search using the database
retrived_search = db.similarity_search(query)

# Extract the page content from the search results
search_results_content = " ".join([doc.page_content for doc in retrived_search])

# Combine the search results into the prompt text
combined_text = f"{query}\n{search_results_content}"

# Invoke the chain with the combined input
response = chain.invoke({"text": combined_text})

# Print the response
print(response)
